<a href="https://colab.research.google.com/github/paulodowd/SEMTM0042_43/blob/main/L4_SystemIntegration_SEMTM0042_43.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labsheet 4: System Integration

**Aims:**

In this labsheet we will learn about:

- Linking discrete or individual behaviours together to create a final solution to Assessment 1: Foraging Challenge.
- Architecting a system, primarily through the construction of a **finite state machine**.
- Debugging a system, primarily through **isolating parts of the system** for testing.


**Objectives:**

Program a **finite state machine** to:
- organise the behaviours of your robot.
- create methodical way to sequence behaviours with respect to changing task requirements.
- implement a way to move between multiple waypoints (x,y coordinates).
- catch errors or exceptional states in your robot operation.
- allow you to isolate parts of your code for debugging and development.
- allow you to easily re-use your code in future work.

**Context:**

At this stage in the labsheets you should have developed run-once routines to calibrate your sensors, code to collect new readings from the sensors, and then **non-blocking** code to allow your robot to move between waypoints (x,y coordinates).  This means that you have all the parts to a solution, but not the solution itself.

It is now important that you attempt to bring these parts together to create your **individual** final solution to the coursework.  












<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/debugging.png" align="left" width=100 > In robotics, **system integration** has a tendency to generate the most errors, bugs and frustration.  There is an anecdotal saying that "the last 20% of a project takes 80% of the time".  Appreciating this idea is very important in robotics.  We can often feel like we have understood all the parts of the problem, but new challenges will exist in integrating the final solution.  In our case, we will meet unexpected challenges because:


- the many sub-components (or sub-systems) may not perform as expected when used with other parts of the system.  
- by addressing the final solution, you will need to **spend much more time watching your robot** perform behaviours.  This means it simply takes more time to locate the errors and bugs.
- you will need to exercise **critical thinking** and **deep understanding** in order to diagnose at which level of the system the problem is occuring.  For example, it could be your high-level understanding of the co-ordinate system that is at fault.  Or, it could be a low-level unhandled error with how you are operating the motors.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> **STOP!** Before you proceed, take some time to tidy up your code.  You are about to encounter some difficult challenges.  You don't want your code to make it more difficult.  Whilst it might feel like a waste of time, you will **save a lot of time** if you create a new copy of your code and:

- remove old code that isn't important any more.
- look for repetition in your code, and move this into a function that can be called instead.
- consider using a **class** to group together variables and functions within a given context.  This will help significantly to avoid ambiguity between local and global variables.
- give all your variables and functions **meaningful names**.  
- **comment your code**. A great way to identify bugs is to explain what your code is doing.
- if you've used `delay()` or `delayMicroseconds()`, see if you can implement a **non-blocking** solution instead.

<hr><br><br><br><br>


# Beginner Difficulty



For the **Beginner Difficulty** category of the coursework, it is not necessary to have your robot move between waypoints.  It is sufficient to demonstrate the robot interacting with the thick black line marking the boundary of the coursework map, and otherwise actively searching the map for 4 minutes.  Therefore, there are 2 principle behaviours you need to organise:
1. Searching
2. Avoiding the Line

These provide a nice way to get a finite state machine working.  We want the robot to alternate between **searching** and **avoiding the line** depending on the reported values of the line sensors.

## Exercise 1: Implementing a Finite State Machine (FSM)

Read the following example code and comments, which is a sparse template of a Finite State Machine to switch between two states in response to the Line Sensors:

```c
// You can use a global variable to represent
// which state (behaviour) your robot is
// operating.  You can use a #define to easily
// add more states, and to make your code
// readable.
#define STATE_DEBUGGING   -1
#define STATE_SEARCHING   0
#define STATE_AVOID_LINE  1
#define STATE_STOP        2
int state;

void setup() {
  //
  // calibrate your sensors, etc
  //


  // Decide which state your robot should
  // start in.
  state = STATE_DEBUGGING;
}


void loop() {

  // Update the robots estimate of position
  pose.update();

  // Update your sensors
  line_sensors.getReadings();
  
  // Check which state we need to be in
  if( line_sensors.readings[0] == ??? ) {
      state = STATE_AVOID_LINE;
  } else {
      state = STATE_SEARCHING;
  }

  // Operate code for which ever state the
  // system is currently in.
  if( state == STATE_AVOID_LINE ) {
      avoidLineBehaviour();

  } else if( state == STATE_SEARCHING ) {
      searchBehaviour();

  } else if( state == STATE_DEBUGGING ) {
      
      // You can keep one state purely for
      // debugging.  For example, you could
      // use this to print out sensor values.
  }

}


```

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left">**Hypothesise:** In the above example code, which state would the robot start in?  Would it ever move to another state?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> In the above example code, notice that at the top of `loop()` the robot first calls `pose.update()` and `line_sensors.getReadings()`.  This follows the best practice of always updating the system state as consistently as possible, and from a single location.  In the example code, this approach would require the functions `avoidLineBehaviour()` and `searchBehaviour()` to also be **non-blocking**.  This would mean these functions would use the most recent systems state information, decide a motor output, and then immediately return (finish).  This would allow `loop()` to run again.  This is the recommended approach to your system design.

<hr><br><br><br><br>


# Intermediate Difficulty

For the Intermediate Difficulty category of the coursework it is highly likely that your robot will need to visit each of the 6 possible puck locations on the coursework map.  If your goal is to push only 1 puck out of the coursework map, you could program your robot to spend the remaining time up to 4 minutes searching the space and avoiding the thick black line marking the boundary of the coursework map (see the previous section, Beginner Difficulty).

However, to achieve higher marks in the Intermediate Difficulty category, your robot will need to visit each of the 6 puck locations.  There are several possible solutions to this **path-planning** or **navigation problem**.



### Boustrophedon Path

One solution to visiting all puck location is to have the robot move up-and-down (or left-to-right) across the coursework map in a systematic manner.  However, there is a clear challenge with this approach.

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/WhiskerEnvelope.png">
</p>

The robot requires the whiskers attached (cable-ties, attached to the bumpers) in order to gain good control of the puck.  However, this also means that the robot can accidentally move the puck location by driving into it.  Once the puck has been moved, it becomes more difficult to find it.  It is true that, if your robot visits all 6 puck locations, your robot can return to the start to randomise a new puck location.  However, this would make the efficiency of your foraging solution very poor.

You may be able to use the Boustrophedon approach with some limited success.  For example, maybe your robot can push out pucks, but only from particular puck locations.  In which case, the performance of your robot becomes reliant on the dice roll used to determine puck location.  This is not desirable.

If you can program your robot to conduct a boustrophedon path, then it is most likely within your skill set to also program some more targeted waypoints.

### Waypoints

The robot can only know if the puck is present or not (true or false).  As described in Labsheet 2, it is most useful to calibrated your magnetometer to only detect the puck when it has entered between the whiskers centre to the robot:

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/catching_puck.png">
</p>

Therefore, a smarter solution would be to attempt to travel to each puck location such that the robot has **good alignment** with the likely position of the puck:

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/WhiskerEnvelope2.png">
</p>

In the above illustration, we can see that the robot first rotated to face location 1, and then moved to keep the centre of the robot (between the whiskers) aligned with location 1.  

Considering the above illustration, it is also important to decide the order of puck locations to visit.  Where there is not a puck at location 1, it is now safe to move over location 1 entirely.  Therefore, rotating close to location 1 will not move a puck.

However, if the robot performs a rotation, it's whiskers could potentially strike a puck (the ciruclar dotted line).  Similarly, whilst the robot is moving, the whiskers extend past the body radius, and may also strike a puck (straight dotted lines).  

For example, moving from Location 1 to Location 3 is likely to push a puck at Location 2 aside.

Therefore, you should decide a strategy of which locations to visit.  Some paths may be more efficient than others.  Note that, with a dice roll, all 6 locations will have equal probability of occuring regardless of any previous locations.

You should also remember that to achieve a higher mark, your robot will need to return to the start to allow you to randomise the next puck location.  Therefore, your robot could either:
- visit all 6 locations unsucessfully, and need to return to the start.
- successfully push a puck out of the map, and need to return to the start.

Therefore, you will likely need to construct your solution with some flexibility about which waypoints are being visited, or in which order.

<hr><br><br><br><br>

## Exercise 2: Implementing Waypoints

To move between waypoints, your robot will need a way to store multiple x,y coordinate values.  An inefficient way to achieve this would be with individual variables, for example:


```c
float x0 =  100;
float y0 =  100;
float x1 =  200;
float y1 = -100;
// ...etc

**A clear problem with this is that you would need to write all of your code like a script** - meaning that, for each x,y pair, your code would need to explictly run some movement code that references those exact variables.  This would cause a lot of duplication of code, and so increase the likelihood of bugs or errors occuring significantly.  For example, you might change a piece of code for one pair of x,y variables, but forget to do it elsewhere.

A much more elegant solution is to use **arrays**, which you will have seen in **Labsheet 2: Sensors**.  Below is a sparse example of how arrays could be used with an index:


```c
// Using global scope so that these values are
// persistent and visible throughout the code.
#define NUM_WAYPOINTS 4
float x[ NUM_WAYPOINTS ] = {0, 100, 200, -100};
float y[ NUM_WAYPOINTS ] = {0, 100, 300,  50 };
int waypoint_index;

void setup() {

  // Start with waypoint 1
  waypoint_index = 1;
}

void loop() {

  // Check if we have arrived at the
  // waypoint. If yes, move to the next
  // waypoint.
  if( travelToXY() == true ) {
    stopRobot();
    waypoint_index = waypoint_index + 1;

    // If we've gone beyond our array of
    // waypoints, we'll start again from the
    // beginning.
    if( waypoint_index >= NUM_WAYPOINTS ) {
      waypoint_index = 0;
    }

  }

}

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> In the above example code, `waypoint_index` is initialised to 1 in `setup()`. Why might this not be 0?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> The above example code intentionally leaves it ambiguous as to whether you could pass the array values `x[]` and `y[]` into your travelToXY() function as arguments, or whether the function makes use of `x[]` and `y[]` as global variables.  What might the advantages and disadvantages to each approach be?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> The above example code catches the potential bug of indexing `x[]` and `y[]` above `NUM_WAYPOINTS`.  If these variables were indexed to a value of `NUM_WAYPOINTS` or larger, what would you expect to see your robot do?


<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** Using the above example code as inspiration or a starting point, implementing your own way to store and access a number of waypoint x,y coordinate pairs.  

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/validate.png" align="left">To start with, implement just 2 pairs of numbers, and see if your robot can reliably move between waypoint 0 and waypoint 1.  You could check that your robot could both stop when moving from waypoint 0 to waypoint 1, and that your robot could loop between waypoint 0, to waypoint 1, back to waypoint 0 again.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/validate.png" align="left"> Extend your number of waypoints, and check that the robot can reliably move between all of them.  Check that, after visiting all the waypoints, the robot can return to the start area.  

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/observation.png" align="left"> **Observations:** Pay careful attention to when your robot is not able to reach a waypoint with the accuracy or precision that you would like.  Try changing the order or sequence of your waypoints, and see if this has an effect.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> At first, you will likely observe that your robot is not able to reach waypoints with the accuracy and precision that you want.  If so, it is important to begin the process of hypothesising how and why this might be.  A core part of the intellectual challenge of this coursework is to analyse and evaluate causes of error in your robotic system.  

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left">  If you did not complete the calibration of your kinematics, you should return to labsheet 3 Motion Control.  If your robot is making sudden changes of movement, oscillating, or you are using open-loop motor control - you may also want to return to labsheet 1: Motors.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/stop.png" align="left"> Pushing your robot causes the gearbox to be **back-driven**.  The gearbox is designed to take a relatively high revolutions per minute (RPM) from the DC motor, and to reduce this down to a relatively low RPM at the wheel.  This **reduction gearing** generates torque at the wheel.  However, if the process is reversed by pushing the robot fast, the energy will move back through the gearbox and strip teeth off the gears - breaking the robot.  **Please be gentle if pushing your robot.  Do not push your robot if the motors are powered.**

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** Once you have your robot moving between multiple waypoints, it is a good idea to try these against the actual possible puck locations of the coursework map.  You have three choices for determining the coordinates of these locations:

1. Using a ruler or tape-measure to physically measure the locations on your coursework map print out.  Remember to then consider the coordinate frame that your robot is using.
2. Use the digital image file in svg format (available on the unit GitHub repository), and measure the distances within a software package such as Inkscape.
3. Use the robot itself to measure the coordinates.  You can leave the battery power off (only plugged in via the USB cable), and ask the robot to report it's position as you move it to a location.  Please be **gentle** with your robot.


<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** Once you have a good enough level of performance for moving between a sequence of waypoints, it is highly recommended you implement a **finite state machine** to manage different behaviours.  An example FSM diagram is given below.  Recall that, for Intermediate Difficulty, your robot will need to:

1. Move between all puck locations to attempt to discover the puck.
2. If it cannot find the puck, it may return to start, and must wait for 4 seconds for a new puck location to be randomised with a dice roll.
3. If it finds the puck, it must push the puck out of the coursework map area.
4. If the puck is pushed out of the map area, the robot must stop with some part of it's white body remaining in the white area.
5. If a puck is successfully pushed out of the map, the robot may return to the start and wait for 4 seconds for a new puck location to be randomised with a dice roll.
6. If a puck is succesfully pushed out of the map, the robot can take the simpler solution of continuing to search the map area for the time remaining within 4 minutes.

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/Intermediate_FSM.png">
</p>

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> In the above FSM diagram, we can observe that if the robot was in the state **Travel to Waypoint** and detected a line, it operates a line avoidance behaviour, and then operates a search behaviour.  Why might a search behaviour be more appropriate than **Set New Waypoint**?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/observation.png" align="left"> Notice that when **Set New Waypoint** exceeds 6, the state machine moves to **Return to Start Area**.  This is an important part of the Assessment Specification.  In order to set a new puck location, your robot must move across all 6 potential puck locations.  For Intermediate Difficulty, the only time this is not true is if it has detected the puck and successfully pushed it out of the coursework map.  In which case, it can return to the start area immediately.

<hr><br><br><br><br>

# Expert Difficulty

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left">  For the Expert Difficulty of the coursework, your starting point should be the ability to complete Intermediate Difficulty with reasonable performance.  You will need code for your robot to move between different locations (waypoints).  **Attempting to solve Expert Difficulty with no prior solution is very difficult and highly discouraged.**  Remember to decompose a problem - break a problem down into smaller parts - and solve each sub-problem in turn.

The below FSM diagram has a small change relative to the Intermediate Difficulty FSM:

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/Expert_FSM.png">
</p>

In the above diagram, we can see that the robot now performs an **Alignment Manoeuvre**, which is the subject of the remainder of this labsheet.  Once the robot has completed this Alignment Manoeuvre, it simply needs to return to the start area, and it should push the puck with it.  This would deposit the puck in the start area.

## Alignment Manoeuvre

In the Expert Difficulty category we have a new challenge.  **The robot must move itself around the puck**, in order to be able to push the puck back to the start area.  Furthermore, **the robot must not accidentally move the puck** whilst it moves around the puck.  If it does accidentally move the puck, it is unlikely the robot will find it again to push it back to the start area.




<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/AlignManeuver.png" width=400>
</p>

The above image helps to illustrate the following important concepts and information available to the robot:
- We know the **whisker envelope**, that is, a radius around the robot where a collision between the whiskers and puck could occur.  Therefore, the robot needs to move _at least_ this distance away.
- We know the **start area co-ordinates**, which will be 0,0 unless you've initialised them differently.
- We will know the **robot coordinates** and rotation when it encounters the puck.
- The robot can estimate the puck **coordinates**, taken from the robot pose and an estimate of distance of detection of the puck.
- Between the puck location and origin, a line can be drawn, which can be thought of as the **push line** (the line of travel of the puck to take it back to the start area).
- Given the push line, we can calculate from the puck location a **end position** of the robot alignment manoeuvre, which is coordinates that the robot needs to get itself to.

With the above information, the requirement becomes to create a series of waypoints to control the robot motion around the puck.  It is important to recognise that for every possible puck location (1-6), and for your approach to searching the coursework map, **it is likely that your alignment manoeuvre will need to be different every time**.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> For the next set of exercises, the challenge will be **significantly easier** if you have written a function like `travelToXY()` that uses a set of target coordinates.  If you have written your code more like a script for navigating puck locations 1-6, it will be difficult to find a general solution to align the robot to push the puck back to the start area.

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/AlignManeuverVariants.png">
</p>

In the above illustration, where **R** is the robot position, **P** is the puck position, and **E** is the end position of the alignment manoeuvre, we can see that:

**(A)**: It is efficient for the robot to move in a clockwise direction around the puck.

**(B)**: It is efficient for the robot to move in an anti-clockwise direction around the puck.

**(C)**: The robot is already behind the puck, so the manoeuvre can be a lot simpler (fewer movements).

**(D)**: The robot may find the puck where there is not a clear efficient direction.

**(A-D)**: Whilst (A),(B) & (C) show the puck at approximately 45 degrees to the origin (start area), this will not always be the case, such as in (D).

For each of **(A)(B)(C)(D)** it is also **very important** that the robot moves itself with sufficient distance from the puck to not push it out of place.  The robot must first of all estimate the puck location, and the return to the puck location to begin pushing it back to the start.  Therefore, if the puck is accidentally moved, pushing the puck back to the start area may fail.

<hr><br><br><br><br>

## Exercise 3: Implementing an Alignment Manoeuvre

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> Like all other exercises in these labsheets, **start simple**.  Decompose the problem and solve each sub-problem in turn.  For your alignment manoeuvre, the first thing to test is how reliably the robot can estimate the puck location.  Observe, hypothesise and validate each step of your alignment manoeuvre.  Start with just one situation (such as **(A)** in the prior illustration) and work to solve that first.  Then test and troubleshoot against **(B)**.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> These final exercises in the Expert Difficulty will challenge you to use **critical thinking**.  For example, there is no reason for you to watch your robot searching the map and detecting a puck.  This would take a long time.  Instead, you can initialise your robot to start at a location, and so setup a **controlled experiment** to develop your alignment manoeuvre.  As you make progress, you can reduce the amount of control you are creating in your experiments.  If something goes wrong, you will want to move your investigation back to a more controlled initial condition.  

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> For the following exercises, be prepared to put aside your general solution to the coursework.  You may need to focus on developing just the sequence of movements for moving around the puck.  Your general coursework solution is probably large, and may confuse the issue.  Once your Alignment Manoeuvre is working well, you can then integrate it back into your general coursework solution.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left"> **Information:** The magnetometer is very sensitive to the presence of other ferrous ("metallic") objects.  For example, a metal table leg is likely to effect the readings of the magnetometer.  If your robot starts to "detect the puck" when it shouldn't, your first thought should be about whether there is a metal object in close proximity.  **This may also include a metal object that was near to the robot temporarily during calibration** - this might include a heavy watch worn on the wrist, or some batteries held in a hand, etc.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** Conduct some experiments with your robot with a focus just on **the reliability of detecting the puck and estimating the puck coordinates**.  To make your life easier, you could start with the robot aligned to the x axis, and the puck positioned on the x axis.  Look to discover:

- A good threshold for the magnetometer reading to detect the puck at a given distance.
- The reliability of the robot stop position to generate a known distance from the puck.
- The relationship between the distance between the robot and puck, and the angle between the robot and the puck.  For example, we don't want to detect the puck when it is not directly ahead of the robot.
- To estimate the puck location, we can use simple trigonometry.  Look to project a point away from the robot's current position at the current rotation angle of the robot, by the estimated distance to the puck.  Remember to sketch (draw) to assist your thinking.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/validate.png" align="left"> Once you have a distance, and you can calculated a projected point ahead of the robot, and instruct your robot to move to that position.  Check that your robot stops where the puck was.   Remember that, whilst testing and developing a solution, you can make the challenge easier.  It might help to draw a large cross and place the puck at the centre.  That way, you can see if your robot moves to the centre of the cross well.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/observation.png" align="left"> **Observations:** At this stage in the coursework, try not to forget all the tools you have available to you for debugging. You can: use `Serial.print()`, display text on the LCD or OLED screen, save variables into the robot memory for later (as global variables), make your robot beep or use it's LEDs to signal different states or events.


<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> In the above exercise, the general guidance is to discover if your current threshold for your magnetometer readings also leads to a reliable stopping distance between your robot and the puck.  It may also be possible to measure the response of the magnetometer at different distance intervals, and so find an equation to describe the relationship to distance.  However, this might be more complicated than it seems.  What other part(s) of the environment might make this more complicated?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** After the above exercise, you should be able to record an estimated location of the puck.  This means you are ready to also create some further waypoints to try to move around the puck.  Before trying many waypoints, take a smaller step.  See if you can set coordinates that are directly behind the puck and on the **push line**.  Once your robot has detected the puck, you can simply move the puck out the way to test if your robot can move to a good **end position** of the alignment manoeuvre.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left"> In the Intermediate Difficult exercises above, the guidance was to create some arrays in global scope for the target position of each puck location, `x[]` and `y[]`.  The guidance also recommended a `#define` to set the maximum array elements (number of coordinate pairs), and an `int` to track which coordinate pair was currently the target. For Expert Difficulty, the simplest approach is to create two more arrays, but to set each coordinate pair with a new value after detecting the puck.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** Once you have verified that your robot can identify a good end position, you can attempt to create some intermediate (in between) waypoints to drive around the puck.  At this point, it would be sensible to implement a second smaller **Finite State Machine** just for the purpose of moving through an Alignment Manoeuvre.  Your robot will need to:


- detect the puck and stop.
- decide which way to navigate around the puck.
- ensure any initial movements do not move the puck (such as an initial rotation).
- know when it has finished the Alignment Manoeuvre.
- transition back to the general FSM for your coursework solution (leave the Alignment Manoeuvre).

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> Remember, there is no reason for you to always watch your robot searching the map and detecting a puck when you are troubleshooting Alignment Manoeuvre problems.  This would take a long time.  Instead, you can initialise your robot to start at a location, and so setup a **controlled experiment**.  You may need to go back to this controlled experiment when a new problem occurs.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> How many extra Alignment Manoeuver waypoints do you think a solution requires?  Will it always be a fixed number?


<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> For the Alignment Manoeuver, an elegantly simple solution might be to have your robot drive on an arc around the puck, until it detects that it has arrived along at a position along the **push line**.  However, this type of **motion control** may have a significant disadvantage relative to the materials provided to you for this coursework.  You are free to innovate and improve on the materials provided to you.  Therefore, what might this issue be?  Identifying the the cause of this issue would be a key area of critical thinking, analysis and evaluation.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> In the guidance given in this labsheet, some emphasis has been made on your Alignment Manoeuvre producing **efficient movement**. Why might this matter in the larger scope of the foraging task?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/validate.png" align="left"> As you progress, it will become necessary to test your solution against the different potential puck locations, and the different stopping positions your robot may discover the puck.  

<hr><br><br><br><br>

# Final Steps

The final steps of this coursework are to improve both the **efficiency** and **reliability** of your solution.  These two characteristics are inter-related.  There is a good chance that if you simply increase the speed of your robot movements (set a higher wheel velocity) to try to make foraging quicker and more efficient, the accuracy and precision of the movements will decrease.  This will in turn reduce the reliability of your system.

In this coursework assignment, your robot is required to operate continuously for 4 minutes.  If we are updating the kinematics at 20millisecond intervals, that is 50 updates per second (50 hertz).  Therefore, across 4 minutes, there will have been 12,000 updates to the kinematics.  That means there were 12,000 opportunities for error to be introduced or accumulated. Your robot is reliant on a good estimate of position to be able to successfully forage for pucks.

As you reach the end of the coursework, it should be that you have gained a **deep understanding** of how each sub-system of your robotic system is operated - you have written your code.  The final stages of this coursework are about the **refinement** of your solution.  To refine your solution, you need to be able to identify where in the system improvements could be made.  

Remember that as roboticists, we will encounter errors, bugs and challenges across **Environment**, **Hardware**, **Software**, and **Task**.  For each of these, try to keep track of your assumptions and your confidence that each is understood and as expected.



<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left">  In the final stages bugs can be very difficult to find because they exist at the level of our human understanding of the **task** - whether we have given (encoded) the correct **domain knowledge** to the robot.  An example of a difficult bug to find and fix would be:

 <blockquote>Your robot detects the puck and immediately moves backwards to avoid pushing it accidentally.  You then estimate the location of the puck from the final position after reversing.  However, your earlier measurements of the distance to the puck were made when your robot detected the puck (prior to the reverse). Therefore, your Alignment Manoeuvre always seems to be a couple of centimeters incorrect.  
 <p align="right">
 A bug experienced by Paul whilst solving the coursework.  It took many hours to find.
 </p>
 </blockquote>

In this example, none of the trigonometry or sequence of calculations would be wrong.  A good way to find bugs like this is to **comment your code** - explain to yourself what the code is attempting to do.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/validate.png" align="left"> A good first refinement is to **double check your calibration of your kinematics parameters** (wheel separation and wheel radius).  It may intuitively feel like any error in the kinematic model will be systematic (repeating, deterministic), and so the only consequence would be a coordinate frame that is slightly skewed (stretched or distorted).  However, in actuality, a poor kinematic model will mean that the displacement of your robot within the coordinate frame is incorrect.  This is because it is information in the local frame that is being transformed into a position in the global frame at every time step.  Therefore, it is not the global frame that presents the error, but how the robot is represented as moving within it.  Your robot will not move in a systematic way for various reasons, but the most obvious is that the puck location is randomised.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left"> **Information:** An easy bug to have relating to the kinematics is to use a `delay()` operation whilst the robot is rotating or moving.  Doing so will mean that the kinematic model is not able to approximate more complex movements.  After the call to `delay()`, it wil receive new encoder counts and most likely assume a large jump in position.  This bug may even impact performance if it is occuring during your run-once sensor calibration.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> **Hypothesise:** In the kinematic model provided there are two parameters, wheel separation and wheel radius.  What is the underlying assumption with only having these two parameters?  Is there a simple way, in terms of model parameters representing the actual robot, that the kinematic model could be improved?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> **Hypothesise:** It is possible for the `loop()` of your code to complete execution within microseconds.  Within this timescale, it is also possible to capture individual encoder counts from each wheel.  This might mean that the kinematics is asked to compute the displacement of the robot with only three possible states of information,$[\Delta E_0=0, \Delta E_1=0]$,  $[\Delta E_0=0, \Delta E_1=1]$, or $[\Delta E_0=1, \Delta E_1=0]$, where $E_0$ is the left encoder, and $E_1$ is the right encoder.  What might be the overall effect of computing the kinematics with this information? What would be important to implement in your code to prevent this?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/observation.png" align="left"> **Observations:** Take the time to look at the motion control of your robot.  An area that may need attention is the ablity of your robot to turn (rotate) a precise angle.  A challenge here is having the robot rotate slowly, which is best solved with PID speed controllers.  

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** If your robot turning behaviour can be improved, consider setting up a small controlled experiment to test and improve your PID speed controllers.  Your main objective will be to achieve steady wheel rotation at low speeds.  This is difficult, so you are advised not to obsess over "smooth motion".  Instead, look to instead gain confidence that the performance you can achieve is good enough.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> **Hypothesise:** Labsheet 1 Motors discussed the issue of extreme input demand values to a position controller.  If you are using a PID heading controller for rotating your robot, you may also encounter extreme input demand values.  For any rotation to a target, the most extreme rotation should be 180°.  However, you can probably achieve much better performance for your heading controller if you instead optimise the PID gains for demand values within a significantly smaller range.  For example the robot will only be asked to rotate within +/-30° by limiting the demand value.  Therefore, you would want to **clamp the input** to your heading controller.  Would doing this have a significant impact on your robot's ability to turn toward a target more than the clamped value?  Furthermore, why might **clamping the output** of the heading controller be a worse solution?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> **Hypothesise:** The general guidance for moving between waypoints has been that your robot should perform two discrete behaviours.  First, rotate on the spot to face the target.  Second, move forwards and make small corrections to rotation.  When the robot is moving forwards and correcting for rotation, there is a relationship between the forward velocity and the impact of rotation correction.  What would be a good motion strategy to improve rotation correction the as the robot nears the target position?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left"> **Information:** There is a relationship between how often you update your PID speed controllers and how often you are estimating the wheel speed.  If your wheel speed estimation is occuring less frequently that your PID update, your PID controllers will be trying to respond to historic (old) information.  

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left"> **Hypothesise:** If you have implemented a low-pass filter to reject noise, it is important to recognise and understand that the filter will introduce delay (lag) into the signal.  We can understand this as an implicit fact quite simply because the (digital) filter must perform some computation on the original signal, which takes time.  What simple adjustment to your code would help to mitigate the effect of lag?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/observation.png" align="left"> **Observations:** Take the time to observe how your robot stops and starts moving. If the motion is jerky (sudden), you may be introducing wheel slip.  This would mean the encoders would count when the robot did not move across the surface.  Even if this slip is very small, repeated wheel slip will accumulate over the 4 minute period.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** To refine your motion control, consider setting up a controlled experiment where you robot will move between two points by moving forwards and then backwards.  For a known distance (e.g., 10cm of travel), check the reported distance after each motion.  A similar controlled experiment can be constructed for rotations between two angles.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/observation.png" align="left"> **Observations:** Take some time to consider the overall motion of your robot.  The kinematics class provided is a minimal implementation that assumes that your robot with either rotate, or translate, but not both.  If both do occur, movement on a curve will be esimated by a straight line approximation.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left">  A very high mark can be achieved on Assessment 1 using only the minimal kinematic model provided.  Within UK higher education, typically marks above 70% are used to celebrate outstanding achievement, often including self-directed study beyond the expectations of the taught materials provided.  Updating the kinematic model to include compound motions is left as an opportunity for your continued study.